# TextGrid to Label Studio

> "Via the API"

- branch: master
- comments: false
- categories: [textgrid, json, hsi]

Update of [an earlier notebook]({% post_url 2024-09-20-textgrid-via-labelstudio-api %}), to include quasi-transcriptions via espeak, and to fix the problem of labels and textareas being separate.

In [1]:
def slurpfile(filename) -> str:
    with open(filename) as inf:
        return inf.read().strip()

In [2]:
host = "http://130.237.3.107:8080/api/"
api_token: str = slurpfile("label_studio_mine")
input_dir = "/Users/joregan/Desktop/"

In [3]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

In [4]:
def get_projects():
    req = requests.get(f"{host}projects", headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [5]:
def get_project_id_from_name(name):
    projects = get_projects()
    for res in projects["results"]:
        if res["title"].strip() == name.strip():
            return res["id"]

In [6]:
def get_tasks(projectid):
    req = requests.get(f"{host}tasks", headers=headers, params={"project": projectid})
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [7]:
def index_task_filestem_to_id(tasks_data):
    tasks = tasks_data["tasks"]
    mapping = {}
    for task in tasks:
        task_id = task["id"]
        if "storage_filename" in task:
            task_raw_path = task["storage_filename"]
        else:
            task_raw_path = task["data"]["audio"]
        if not task_raw_path:
            continue
        task_stem = task_raw_path.split("/")[-1]
        mapping[task_stem] = task_id
    return mapping

In [43]:
tasks = get_tasks(8)

In [44]:
mapping = index_task_filestem_to_id(tasks)

In [16]:
import json
from praatio import textgrid
import uuid

noises = ["smack", "spn", "mic_click", "labial_trill", "sniff", "click", "vocal_clicks", "throat_noise", "vocal_noise", "lip_trill", "lip_noise", "noise", "flapping_noises", "cough", "grumble", "skip", "creak", "s"]
breath = ["breath", "inhale", "exhale", "sigh", "breath_noise", "breath_noises", "blow", "suck"]
laugh = ["laugh", "laughter"]

labels = {}
for noise in noises:
    labels[noise] = "Noise"
for noise in laugh:
    labels[noise] = "Laughter"
for noise in breath:
    labels[noise] = "Breath"

def tg_to_result(tgfile):
    outputs = []
    tg = textgrid.openTextgrid(tgfile, False)

    espeak = {}
    espeak_tier = "espeak"
    es_tier = tg.getTier(espeak_tier)
    for entry in es_tier.entries:
        text = entry.label.strip()
        if text == "":
            continue
        espeak[entry.start] = text

    tiername = "utterances"
    if not tiername in tg.tierNames:
        tiername = "words"

    tier = tg.getTier(tiername)
    for entry in tier.entries:
        text = entry.label.strip()
        if text == "":
            continue

        label = "Speech"
        if text.endswith("crosstalk]"):
            label = "Cross-talk"
            if text.startswith("[") and text.endswith("]"):
                text = text[1:-1]
        elif text.startswith("[") and text.endswith("]") and text[1:-1] in labels:
            label = labels[text[1:-1]]
            text = text[1:-1]
        
        if entry.start in espeak:
            estext = f"/{espeak[entry.start]}/"
            atext = [text, estext]
        else:
            atext = [text]

        gen_id = str(uuid.uuid4())[:6]
        segment = {
            "value": {
                "start": entry.start,
                "end": entry.end,
                "channel": 0,
                "labels": [label]
            },
            "from_name": "labels",
            "to_name": "audio",
            "type": "labels",
            "id": gen_id,
        }
        rec = {
            "value": {
                "start": entry.start,
                "end": entry.end,
                "channel": 0,
                "text": atext
            },
            "from_name": "transcription",
            "to_name": "audio",
            "type": "textarea",
            "id": gen_id,
        }
        outputs.append(segment)
        outputs.append(rec)

    return outputs

In [17]:
def post_results(id, task, project, results):
    ep = f"{host}annotations/{id}/?taskID={task}&project={project}"

    cur_headers = {i: headers[i] for i in headers}
    cur_headers["Content-type"] = "application/json"

    content = {
        "was_cancelled": False,
        "ground_truth": False,
        "project": project,
        "draft_id": 0,
        "parent_prediction": None,
        "parent_annotation": None,
        "result": results
    }
    r = requests.patch(ep, data=json.dumps(content), headers=cur_headers)
    return r

In [18]:
file = f"{input_dir}hsi_4_0717_209_002_main.TextGrid"
data = tg_to_result(file)

In [19]:
r = post_results(270, 72, 5, data)
print(r.text)

{"id":270,"result":[{"value":{"start":5.804032706613903,"end":9.021769689180255,"channel":0,"labels":["Cross-talk"]},"from_name":"labels","to_name":"audio","type":"labels","id":"a4995c"},{"value":{"start":5.804032706613903,"end":9.021769689180255,"channel":0,"text":["crosstalk"]},"from_name":"transcription","to_name":"audio","type":"textarea","id":"a4995c"},{"value":{"start":9.021769689180255,"end":13.893398836807094,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels","id":"587b96"},{"value":{"start":9.021769689180255,"end":13.893398836807094,"channel":0,"text":["Well, yeah, I think you maybe have to look at the fingers. Because in the pinky fingers.","/wˈʌ jˈæ aɪ θˈɪŋk juː mˈeɪbiː æv tə lˈʊk æ ðə fˈɪŋɡɚz. ks ɪn ðə pˈɪŋki fˈɪŋɡɚz./"]},"from_name":"transcription","to_name":"audio","type":"textarea","id":"587b96"},{"value":{"start":14.487499775581218,"end":17.20729204576262,"channel":0,"labels":["Cross-talk"]},"from_name":"labels","to_name":"audio","ty